In [1]:
!pip install supervision Pillow ultralytics numpy opencv-python inference-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.8/898.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.9
    Uninstalling aiohttp-3.11.9:
      Successfully uninstalled aiohttp-3.11.9


In [15]:
import os
import cv2
import json
import numpy as np
import supervision as sv
from ultralytics import YOLO
from inference_sdk import InferenceHTTPClient

# Initialize models and components
vehicle_model = YOLO("yolov8n.pt")  # YOLOv8 for vehicle detection


# Define output video path
video_path = "/content/test.mp4"
output_video_path = "/content/output.mp4"
output_fps = 30  # Adjust this based on your input video's frame rate
frame_size = None  # Will be set dynamically based on the first frame

# Initialize the video writer
video_writer = None

client = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="L5uh8sOgmpA0sglsoQzM"  # Replace this with your actual API key
)

license_plate_model_id = "license-plate-recognition-rxg4e/6"

# Tracking and annotators
tracker = sv.ByteTrack()
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# Vehicle classes (COCO IDs)
VEHICLE_CLASSES = {2: "car", 5: "bus", 7: "truck"}

frames_generator = sv.get_video_frames_generator(video_path)

In [16]:
def matricule(frame):
    try:
        # Perform inference on the frame
        result = client.infer(frame, model_id=license_plate_model_id)
        return result
    except Exception as e:
        print(f"Error during inference: {e}")
        return None

In [17]:
# Callback function for vehicle detection
def process_vehicles(frame: np.ndarray) -> np.ndarray:
    results = vehicle_model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    mask = np.isin(detections.class_id, list(VEHICLE_CLASSES.keys()))
    detections = detections[mask]
    detections = tracker.update_with_detections(detections)

    labels = [
        f"#{tracker_id} {VEHICLE_CLASSES[class_id]}"
        for class_id, tracker_id in zip(detections.class_id, detections.tracker_id)
    ]
    annotated_frame = box_annotator.annotate(frame.copy(), detections=detections)
    return label_annotator.annotate(annotated_frame, detections=detections, labels=labels)

In [18]:
for idx, frame in enumerate(frames_generator):
    # Process vehicles on the frame
    annotated_frame = process_vehicles(frame)

    # Perform license plate recognition
    try:
        result = client.infer(frame, model_id=license_plate_model_id)
        if result and "predictions" in result:
            predictions = result["predictions"]
            for prediction in predictions:
                # Extract license plate bounding box
                x, y, width, height = prediction["x"], prediction["y"], prediction["width"], prediction["height"]
                confidence = prediction["confidence"]
                label = prediction["class"]

                # Convert center-based box to corner-based coordinates
                left = int(x - width / 2)
                top = int(y - height / 2)
                right = int(x + width / 2)
                bottom = int(y + height / 2)

                # Draw license plate bounding box
                cv2.rectangle(annotated_frame, (left, top), (right, bottom), (0, 0, 255), 2)
                cv2.putText(
                    annotated_frame,
                    f"{label} ({confidence:.2f})",
                    (left, top - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0, 0, 255),
                    2,
                )
    except Exception as e:
        print(f"Error processing license plate on frame {idx}: {e}")

    # Initialize the video writer if not done yet
    if video_writer is None:
        frame_size = (annotated_frame.shape[1], annotated_frame.shape[0])  # (width, height)
        video_writer = cv2.VideoWriter(
            output_video_path,
            cv2.VideoWriter_fourcc(*"mp4v"),
            output_fps,
            frame_size
        )

    # Write annotated frame to output video
    video_writer.write(annotated_frame)

# Release the video writer
if video_writer:
    video_writer.release()

print("Video processing complete. Saved to:", output_video_path)


0: 384x640 22 cars, 2 buss, 2 trucks, 13.4ms
Speed: 2.4ms preprocess, 13.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 2 buss, 2 trucks, 8.6ms
Speed: 5.0ms preprocess, 8.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 2 trucks, 9.5ms
Speed: 3.2ms preprocess, 9.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 25 cars, 1 bus, 2 trucks, 14.2ms
Speed: 3.2ms preprocess, 14.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 26 cars, 1 bus, 2 trucks, 9.7ms
Speed: 7.6ms preprocess, 9.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 3 trucks, 11.4ms
Speed: 3.1ms preprocess, 11.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 3 trucks, 9.4ms
Speed: 3.3ms preprocess, 9.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 